# Homework Grading
The goal of this is to develop a robust homework grading system which combines Google Colab + Jupyter Notebooks + GitHub Classroom + Otter Grader + Blackboard. 

### Setup


  1. Make sure that Docker is installed and the Docker Daemon is running.  You can download Docker [here](https://docs.docker.com/get-docker/).

  2. Clone the `Otter Helper` repository from [here](https://github.com/jkuruzovich/otter_helper).
  
  3.  Download assignment from Blackboard or GitHub and put it in the `assignments/<course>/<assignment>` directory. If you then paste your solution in this directory, it will be a good test that it was possible to get 100% on all of the tests.
  
  4. Update the `/config/course/<course>.yml` file with the appropriate configuration and assignments. 
  
  5. Verify that the appropriate tests and data are in the associated `config/course/assignment/` folder. 
  
  6. Verify that the appropriate files specified in the assignments are in the associated `config/course/files/` folder.
  
  7. Verify that the roster is setup in `config/course/roster.xlsx`. This includes 1 sheet which maps GitHub id to student id (if using GitHub classroom) and 1 that is a template downloaded from Blackboard. 
  
  8. Update set the `course` and `assignment_id` varaibles in the notebook to be consistent with the paths files downloaded and the `config.yml` file. To run the sample assignments, just keep `course` set to `sample-class` and select `blackboard` or `github-classroom` for the `assignment_id`.

**After completing the above, you are ready to launch Notebook and run the steps of the notebook below.**  


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#set to what you want to grade
course = 'sample-class' #Sample assignments. 
assignment_id = 'blackboard'  #Choose 'blackboard' or 'github-classroom' for samples. 


### Build the Docker Container
I find it is best to customize your docker container and build it if grading locally. If you add your requirements for a single class docker container and not to each assignment, overall grading will go much faster. 

Uncomment the cells below and run it. Be sure to keep the `!` as this is run via the command line. 

In [3]:
#!cd ../docker && docker build . -t otter-helper:latest

### Set and Load the Configuration
This loads the configuration from the `/config/course/config.yml` file.  

In [4]:
from pathlib import Path
import configparser, sys, os, importlib
import pandas as pd
 
cwd_dir = Path.cwd() #For running locally
base_dir = cwd_dir.parent #For running locally
#base_dir='/content/drive/My Drive/autograding'
modules_path = base_dir / 'modules' 
sys.path.append('../modules') # Not sure why appending the mudles path didn't work. 

#Load the autograding library
import autograde as ag
importlib.reload(ag)
cf = ag.get_config(course, assignment_id, base_dir)
cf

{'class_name': 'Sample Class',
 'class_id': 'sample-class',
 'message_complete': 'Your submission was successfully received and graded.<br>',
 'message_incomplete': 'If you get this message it is because you did not submit assignment. Please see the TA.<br>',
 'num_containers': 4,
 'ignore': ['.ipynb_checkpoints', '.DS_Store'],
 'requirements': 'config/sample-class/requirements.txt',
 'roster': 'config/sample-class/roster.xlsx',
 'assignments': {'blackboard': {'name': 'Sample assignment',
   'type': 'bb',
   'extension': '.ipynb',
   'tests_path': 'config/sample-class/blackboard/hidden-tests',
   'requirements': 'config/sample-class/requirements.txt',
   'seed': 42,
   'bb_column': 'blackboard_grade_test',
   'files': ['config/files/test_data_a.csv', 'config/files/test_data_b.csv']},
  'github-classroom': {'name': 'Sample assignment2',
   'type': 'gc',
   'extension': '.ipynb',
   'tests_path': 'config/sample-class/github-classroom/hidden-tests',
   'requirements': 'config/sample-class

### Prepare Grading
The will copy the assignent files either from the Blackboard or the GitHub classroom files in the `/assignments/course/assignment` folder to the `/tmp/` folder. It will also generate the needed `meta.json` file which maps the identifier (GitHub/Student ID) to the submission file. 

*For BlackBoard*
This makes the assumption that for files collected via Blackboard, each assignment file will be in the form `assignment_identifier_submissiondate`. The identifier is pulled out.  If your pattern is different, minor changes might be needed. [TBD, include a pattern in config.]

*For Github*
This makes the assumption that for files collected via Github Classroom, each assignment is in a different directory, where the directory name is the student's GitHub id. This GitHub id is used as the identifier for grading and matched with the student id after grading. 

The `cleaup=True` option just deletes the tmp folder before starting and is recommended.  

In [5]:
files=ag.prepare_grade(cf,  cleanup=True)  
files

[{'identifier': 'fails3', 'filename': 'assignment_fails3_attempt_2020.ipynb'},
 {'identifier': 'fails1Hidden',
  'filename': 'assignment_fails1Hidden_attempt_2020.ipynb'},
 {'identifier': 'passesAll',
  'filename': 'assignment_passesAll_attempt_2020.ipynb'},
 {'identifier': 'fails2Hidden',
  'filename': 'assignment_fails2Hidden_attempt.ipynb'},
 {'identifier': 'fails3Hidden',
  'filename': 'assignment_fails3Hidden_attempt.ipynb'},
 {'identifier': 'fails1', 'filename': 'assignment_fails1_attempt_2020.ipynb'},
 {'identifier': 'fails2', 'filename': 'assignment_fails2_attempt_2020.ipynb'}]

### Running the Grader 

This will launch several Docker containers (the number specified by the `config.yml` file, grade the assignment, and aggregate the assocated grades in the `./tmp/final_grades.csv`.

In [6]:
!cd ../tmp && otter -j meta.json -t tests --image otter-helper:latest -v

Found JSON metadata...
Launching docker containers...
Launched container d48a546c6c8e...
Launched container 74b97f215c1c...
Launched container 0efe6e98f226...
Launched container bdf2fb397747...
Installing requirements in container d48a546c6c8e...
Installing requirements in container bdf2fb397747...
Installing requirements in container 0efe6e98f226...
Installing requirements in container 74b97f215c1c...
Grading notebooks in container d48a546c6c8e...
Grading notebooks in container bdf2fb397747...
Grading notebooks in container 0efe6e98f226...
Grading notebooks in container 74b97f215c1c...
Copying grades from container d48a546c6c8e...
Copying grades from container 0efe6e98f226...
Stopping container d48a546c6c8e...
Stopping container 0efe6e98f226...
Copying grades from container 74b97f215c1c...
Copying grades from container bdf2fb397747...
Stopping container 74b97f215c1c...
Stopping container bdf2fb397747...
Combining grades and saving...


### Review the Grades
No you have graded all the students there is an output file in `./tmp/final_grades.csv`.  If you have added your solution to this, it is a good practice to double check that your solution passed all of the associate tests. 

If you find that nearly all students missed a test, it may also signal an issue with the test. You can always adjust the tests and then re-run the above grading. 


In [7]:
grades = pd.read_csv(cf['grade_file'])
grades

,identifier,file,q1,q1H,q2,q2H,q3,q3H,total,possible
0,fails2,assignment_fails2_attempt_2020.ipynb,1.0,2.0,0.0,0.0,1.0,2.0,6.0,8
1,fails2Hidden,assignment_fails2Hidden_attempt.ipynb,1.0,2.0,1.0,0.0,1.0,2.0,7.0,8
2,fails1,assignment_fails1_attempt_2020.ipynb,0.0,0.0,1.0,1.0,1.0,2.0,5.0,8
3,fails3Hidden,assignment_fails3Hidden_attempt.ipynb,1.0,2.0,1.0,1.0,1.0,0.0,6.0,8
4,passesAll,assignment_passesAll_attempt_2020.ipynb,1.0,2.0,1.0,1.0,1.0,2.0,8.0,8
5,fails1Hidden,assignment_fails1Hidden_attempt_2020.ipynb,1.0,0.0,1.0,1.0,1.0,2.0,6.0,8
6,fails3,assignment_fails3_attempt_2020.ipynb,1.0,2.0,1.0,1.0,0.0,0.0,5.0,8


### Archive `tmp` Directory and Generate the Upload to Blackboard File

This will archive the `/tmp` directory to the `/output/course/assignment` directory so that you have a persistent copy of the final version.   


This will loop through the students in your roster file (Blackboard tab) and match it with any graded submissions.  For GitHub classroom it will also match the github ID with the student ID. It will then generate the `/output/course/assignment/upload.csv` file which is ready to be uploaded to blackboard. Scores on each test are included in the `Feedback to Learner` column along with the message included in the `config.yml` file.

**It is important to make sure that your `roster.xslx` file is up to date. If a student entered the class late and isn't on the roster, that student's grade won't be updated.**

**It is important to make sure that the `bb_column` in the `config.yml` file matches the column in Blackboard when you download for offline grading.**



In [8]:
#Aggregate JSON files for manual grading. 
blackboard=ag.prepare_blackboard_upload(cf, archive=True)
blackboard

complete: 6 
Incomplete: 0 
Total: 6
Archiving files in 


,Last Name,First Name,Username,Student ID,Last Access,Availability,blackboard_grade_test,Grading Notes,Notes Format,Feedback to Learner,Feedback Format
0,Name,Name,fails3,fails3,2020-08-01 22:08:00,Yes,5.0,NaN,NaN,Your submission was successfully received and ...,HTML
1,Name,Name,passesAll,passesAll,2020-08-01 22:08:00,Yes,8.0,NaN,NaN,Your submission was successfully received and ...,HTML
2,Name,Name,fails2Hidden,fails2Hidden,2020-08-01 22:08:00,Yes,7.0,NaN,NaN,Your submission was successfully received and ...,HTML
3,Name,Name,fails3Hidden,fails3Hidden,2020-08-01 22:08:00,Yes,6.0,NaN,NaN,Your submission was successfully received and ...,HTML
4,Name,Name,fails2,fails2,2020-08-01 22:08:00,Yes,6.0,NaN,NaN,Your submission was successfully received and ...,HTML
5,Name,Name,fails1,fails1,2020-08-01 22:08:00,Yes,5.0,NaN,NaN,Your submission was successfully received and ...,HTML


## Upload to Blackboard File 

TBD: Flag student submissions which aren't in the roster file.

Upload the `/output/course/assignment/upload.csv` and you are done. 